## 3. Neural Network

#### Importing libraries

In [6]:
import torch
import torch.nn as nn 
import torch.nn.functional as f

#### Table of content

* [Introduction and random notes](#notes)
* [Define the network](#definition)

#### Introduction and random notes

Neutral Networds can be constructed using the torch.nn package
    - Depends on autograd to define models and differentiate them 

An nn.Module contains: 
    - layers 
    - A method: 
        - forward(input) that returns the output

#### Example: MNIST - A network that classifies digit images

<img src="../images/beginner/mnist.png"></img>

#### A typical training procedure 

1. Define the neural network that has some learnable parameters (or weights)
2. Iterate over a dataset of inputs
3. Process input through the network 
4. Compute the loss (how far is the output from being correct) 
5. Propagate gradients back into the network's parameters 
6. Update the weights of the network, typically using a simple update rule: 
    - Weight = weight - learning_rate * gradients (Gradient Descent)

## Define the network <a class="anchor" id="definition"></a>

In [ ]:
class Net(nn.Module): # Our definition is the child of the parent class nn.module
    
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1,6,5) # 1 input image channel, 6 output channels, 5*5 square convolution kernel
        self.conv2 = nn.Conv2d(6,16,5) # 6 input channels
        
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    
    def forward(self, x):
        